# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Event-Driven Architecture

**Subtopic** - Implementing a Simple Event-Driven System

**Use Case** - Design a FastAPI endpoint that asynchronously publishes incoming HTTP requests as events to a message broker, adhering to event-driven API design principles.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

Implement a production-ready event-driven order processing system that handles concurrent order processing with proper async patterns, race condition prevention, and exactly-once event processing guarantees. The system must process orders through multiple states while maintaining data consistency and handling failures gracefully.

Your solution should be contained in single file and must have driver function. It must run on an online python compiler and must be testable.

## Input Format

```
# POST /process_order
{
    "customer_data": {
        "name": str,
        "email": str,
        "address": str
    },
    "items": [
        {
            "name": str,
            "price": float,
            "quantity": int
        }
    ]
}

# GET /orders/{order_id}/status
# No body required

# GET /orders/{order_id}/events  
# No body required
```

## Output Format

```
# POST /process_order response
{
    "success": bool,
    "order_id": str,
    "message": str,
    "timestamp": str (ISO format)
}

# GET /orders/{order_id}/status response
{
    "success": bool,
    "order": {
        "order_id": str,
        "status": str,  # pending|validated|payment_processed|shipped|completed|failed
        "created_at": str,
        "updated_at": str,
        "total_amount": float
    }
}

# GET /orders/{order_id}/events response
{
    "success": bool,
    "events": [
        {
            "event_type": str,
            "timestamp": str,
            "data": dict,
            "processed": bool
        }
    ]
}
```

## Examples

Example 1:

```
input_data = {
    "customer_data": {
        "name": "John Doe",
        "email": "john@example.com",
        "address": "123 Main St"
    },
    "items": [
        {"name": "Laptop", "price": 999.99, "quantity": 1}
    ]
}

expected_flow = [
    "ORDER_CREATED",
    "ORDER_VALIDATED",
    "PAYMENT_PROCESSED",
    "ORDER_SHIPPED",
    "ORDER_COMPLETED"
]
```

Example 2:

```
# Concurrent order processing
# Submit 10 orders simultaneously
# All should be processed without race conditions
# Each order should follow the complete state machine
```

# Requirements

## Explicit Requirements

1. Event-Driven Architecture: Use publish-subscribe pattern with async message handling
2. State Machine: Orders must progress: pending -> validated -> payment_processed -> shipped -> completed
3. Concurrency Control: Prevent race conditions using proper locking mechanisms
4. Exactly-Once Processing: Each event processed exactly once, even if duplicated
5. Error Handling: Implement compensation logic for failed orders
6. Resource Management: Proper cleanup of async resources and failed orders
7. Testability: Use dependency injection for time and external dependencies
8. Production Patterns: Implement retry logic, circuit breakers, and health checks

## Implicit Requirements

1. Thread Safety: All shared state must be properly synchronized
2. Memory Management: No memory leaks from unclosed async tasks/queues
3. Event Ordering: Critical events must be processed in correct sequence
4. Idempotency: Duplicate events should be safely ignored
5. Graceful Degradation: System continues working even with partial failures
6. Monitoring: Proper logging with correlation IDs for debugging
7. Scalability: Handle high concurrent load without performance degradation
8. Data Consistency: No partial state updates or orphaned records

## Function Signature

```python
class OrderProcessingSystem:
    def __init__(self, time_provider: TimeProvider = None):
        """Initialize with optional time provider for testing"""
        pass
    
    async def process_order(self, customer_data: dict, items: List[dict]) -> dict:
        """Process a new order and return order ID"""
        pass
    
    async def get_order_status(self, order_id: str) -> dict:
        """Get current order status"""
        pass
    
    async def get_order_events(self, order_id: str) -> dict:
        """Get all events for an order"""
        pass
    
    async def get_health(self) -> dict:
        """Get system health status"""
        pass
    
    async def startup(self):
        """Initialize the system"""
        pass
    
    async def shutdown(self):
        """Gracefully shutdown the system"""
        pass
```

## Edge Cases

1. Duplicate Events: Same event published multiple times
2. Out-of-Order Events: Events arriving in wrong sequence
3. Concurrent Updates: Multiple handlers updating same order simultaneously
4. Handler Failures: Event handlers throwing exceptions
5. System Shutdown: Graceful shutdown with pending events
6. Resource Exhaustion: High load causing memory/CPU issues
7. Invalid State Transitions: Attempting invalid status changes
8. Orphaned Orders: Orders stuck in intermediate states
9. Event Handler Timeouts: Long-running handlers exceeding timeouts
10. Network Partitions: Simulated failure scenarios

## Constraints

- Performance: Process 100 concurrent orders without data corruption
- Memory: No memory leaks from async resources
- Timing: Event processing must complete within 30 seconds
- Reliability: 99.9% event processing success rate
- Consistency: Zero data inconsistency under concurrent load
- Testability: All components must be unit testable
- Error Recovery: Failed orders cleaned up within 30 seconds
- Scalability: Linear performance scaling with worker count
- Observability: All critical events must be logged with correlation IDs
- Production Ready: Handle all edge cases gracefully without crashes


In [ ]:
# code

"""
Single-file, production-ready, event-driven order processing system in Python
with concurrency, exactly-once event processing, state machine transitions,
error handling, and test/demo driver function.
"""

import asyncio
import uuid
import random
from datetime import datetime
from typing import List, Dict, Any, Optional


class TimeProvider:
    """
    Simple time provider that cycles through a fixed list of ISO timestamps.
    This avoids using datetime.now() or any dynamic system time calls for
    predictable testing and deterministic behavior.
    """

    def __init__(self, times: Optional[List[str]] = None):
        self._times = times or [
            f"2023-01-01T00:{str(i).zfill(2)}:00Z" for i in range(21)
        ]
        self._index = 0

    def get_current_time(self) -> str:
        """Return a fixed ISO8601 string from the internal list."""
        current_time = self._times[self._index % len(self._times)]
        self._index += 1
        return current_time


class SystemTimeProvider:
    """Alternative time provider that uses actual system time."""

    def now(self) -> datetime:
        return datetime.now()


class OrderProcessingSystem:
    """
    Main order processing system that handles order lifecycle through
    event-driven state transitions with concurrency control.
    """

    MAX_PAYMENT_RETRIES = 3

    def __init__(self, time_provider: Optional[TimeProvider] = None):
        self.time_provider = time_provider or TimeProvider()
        self._orders: Dict[str, Dict[str, Any]] = {}
        self._order_events: Dict[str, List[Dict[str, Any]]] = {}
        self._processed_event_ids: set = set()
        self._order_locks: Dict[str, asyncio.Lock] = {}
        self._event_queue: asyncio.Queue = asyncio.Queue()
        self._shutdown_flag = False
        self._event_loop_task: Optional[asyncio.Task] = None
        self._payment_retry_count: Dict[str, int] = {}

    async def process_order(self, customer_data: dict,
                            items: List[dict]) -> dict:
        order_id = str(uuid.uuid4())
        created_at = self.time_provider.get_current_time()
        total_amount = sum(item['price'] * item['quantity'] for item in items)

        self._orders[order_id] = {
            "order_id": order_id,
            "status": "pending",
            "created_at": created_at,
            "updated_at": created_at,
            "customer_data": customer_data,
            "items": items,
            "total_amount": total_amount,
        }

        self._order_events[order_id] = []
        self._order_locks[order_id] = asyncio.Lock()
        self._payment_retry_count[order_id] = 0

        await self._publish_event(
            event_type="ORDER_CREATED",
            order_id=order_id,
            data={"reason": "New order created"}
        )

        return {
            "success": True,
            "order_id": order_id,
            "message": "Order created successfully",
            "timestamp": created_at,
        }

    async def get_order_status(self, order_id: str) -> dict:
        order = self._orders.get(order_id)
        if not order:
            return {"success": False, "order": {}}

        return {
            "success": True,
            "order": {
                "order_id": order["order_id"],
                "status": order["status"],
                "created_at": order["created_at"],
                "updated_at": order["updated_at"],
                "total_amount": order["total_amount"],
            }
        }

    async def get_order_events(self, order_id: str) -> dict:
        if order_id not in self._order_events:
            return {"success": False, "events": []}
        return {"success": True, "events": self._order_events[order_id]}

    async def get_health(self) -> dict:
        return {
            "success": True,
            "status": "shutdown" if self._shutdown_flag else "healthy"
        }

    async def startup(self):
        self._shutdown_flag = False
        self._event_loop_task = asyncio.create_task(
            self._event_consumer_loop())

    async def shutdown(self):
        self._shutdown_flag = True
        if self._event_loop_task:
            while not self._event_queue.empty():
                await asyncio.sleep(0.01)
            self._event_loop_task.cancel()
            try:
                await self._event_loop_task
            except asyncio.CancelledError:
                pass

    async def validate_order(self, order_id: str) -> bool:
        order = self._orders.get(order_id)
        if not order:
            return False
        if not order.get("items"):
            raise Exception("No items in order")
        for item in order["items"]:
            if item.get("price", 0) <= 0:
                raise Exception("Invalid item price")
        return True

    async def process_payment(self, order_id: str) -> bool:
        return await self._attempt_payment(order_id)

    async def _publish_event(
            self, event_type: str, order_id: str, data: dict = None):
        event_id = str(uuid.uuid4())
        timestamp = self.time_provider.get_current_time()
        event = {
            "event_id": event_id,
            "event_type": event_type,
            "order_id": order_id,
            "timestamp": timestamp,
            "data": data or {}
        }
        await self._event_queue.put(event)
        if order_id in self._order_events:
            self._order_events[order_id].append({
                "event_type": event_type,
                "timestamp": timestamp,
                "data": data or {},
                "processed": False
            })

    async def _mark_event_processed(self, order_id: str, event_id: str):
        if order_id not in self._order_events:
            return
        for event in self._order_events[order_id]:
            if not event.get("processed", False):
                event["processed"] = True
                break

    async def _event_consumer_loop(self):
        while not self._shutdown_flag or not self._event_queue.empty():
            try:
                event = await asyncio.wait_for(
                    self._event_queue.get(), timeout=0.1)
            except asyncio.TimeoutError:
                continue

            event_id = event["event_id"]
            if event_id in self._processed_event_ids:
                self._event_queue.task_done()
                continue

            self._processed_event_ids.add(event_id)
            order_id = event["order_id"]
            event_type = event["event_type"]
            data = event["data"]

            try:
                await self._handle_event(event_id, order_id, event_type, data)
            except Exception as ex:
                print(f"[ERROR] Event handling failed: {ex}")
                if order_id in self._orders:
                    await self._on_order_failed(order_id, {"reason": str(ex)})
            finally:
                self._event_queue.task_done()

    async def _handle_event(
            self, event_id: str, order_id: str, event_type: str, data: dict):
        lock = self._order_locks.get(order_id)
        if not lock or order_id not in self._orders:
            return
        async with lock:
            if order_id not in self._orders:
                return
            if event_type == "ORDER_CREATED":
                await self._on_order_created(order_id)
            elif event_type == "ORDER_VALIDATED":
                await self._on_order_validated(order_id)
            elif event_type == "PAYMENT_PROCESSED":
                await self._on_payment_processed(order_id)
            elif event_type == "ORDER_SHIPPED":
                await self._on_order_shipped(order_id)
            elif event_type == "ORDER_FAILED":
                await self._on_order_failed(order_id, data)
            await self._mark_event_processed(order_id, event_id)

    async def _on_order_created(self, order_id: str):
        order = self._orders[order_id]
        try:
            await self.validate_order(order_id)
            order["status"] = "validated"
            order["updated_at"] = self.time_provider.get_current_time()
            await self._publish_event(
                event_type="ORDER_VALIDATED",
                order_id=order_id,
                data={"reason": "Validation successful"}
            )
        except Exception as e:
            await self._publish_event(
                event_type="ORDER_FAILED",
                order_id=order_id,
                data={"reason": f"Validation failed: {str(e)}"}
            )

    async def _on_order_validated(self, order_id: str):
        order = self._orders[order_id]
        order["status"] = "payment_processing"
        order["updated_at"] = self.time_provider.get_current_time()
        try:
            if await self.process_payment(order_id):
                await self._publish_event(
                    event_type="PAYMENT_PROCESSED",
                    order_id=order_id,
                    data={"reason": "Payment OK"}
                )
            else:
                await self._publish_event(
                    event_type="ORDER_FAILED",
                    order_id=order_id,
                    data={"reason": "Payment failed"}
                )
        except Exception as e:
            await self._publish_event(
                event_type="ORDER_FAILED",
                order_id=order_id,
                data={"reason": f"Payment error: {str(e)}"}
            )

    async def _on_payment_processed(self, order_id: str):
        order = self._orders[order_id]
        order["status"] = "shipped"
        order["updated_at"] = self.time_provider.get_current_time()
        await self._publish_event(
            event_type="ORDER_SHIPPED",
            order_id=order_id,
            data={"reason": "Order shipped"}
        )

    async def _on_order_shipped(self, order_id: str):
        order = self._orders[order_id]
        order["status"] = "completed"
        order["updated_at"] = self.time_provider.get_current_time()
        await self._publish_event(
            event_type="ORDER_COMPLETED",
            order_id=order_id,
            data={"reason": "Order flow complete"}
        )

    async def _on_order_failed(self, order_id: str, data: dict):
        order = self._orders.get(order_id)
        if not order:
            return
        order["status"] = "failed"
        order["updated_at"] = self.time_provider.get_current_time()

    async def _attempt_payment(self, order_id: str) -> bool:
        success_chance = 0.9
        attempt_number = self._payment_retry_count.get(order_id, 0)
        self._payment_retry_count[order_id] = attempt_number + 1
        if attempt_number >= self.MAX_PAYMENT_RETRIES:
            return False
        if random.random() < success_chance:
            return True
        await asyncio.sleep(0.01)
        await self._publish_event(
            event_type="ORDER_VALIDATED",
            order_id=order_id,
            data={"reason": f"Payment attempt {attempt_number + 1} failed,\
                   scheduling retry"}
        )
        return False


async def driver():
    custom_times = [
        f"2023-01-01T12:00:{str(i).zfill(2)}Z" for i in range(12)
    ]
    time_provider = TimeProvider(times=custom_times * 50)
    system = OrderProcessingSystem(time_provider=time_provider)
    await system.startup()

    print("Creating a single order:")
    order_data = {
        "customer_data": {
            "name": "John Doe",
            "email": "john@example.com",
            "address": "123 Main St"
        },
        "items": [
            {"name": "Laptop", "price": 999.99, "quantity": 1}
        ]
    }
    response = await system.process_order(
        order_data["customer_data"], order_data["items"])
    print("process_order response:", response)
    order_id = response["order_id"]
    await asyncio.sleep(0.5)
    status = await system.get_order_status(order_id)
    print("Order Status after 0.5s:", status)
    events = await system.get_order_events(order_id)
    print("Order Events after 0.5s:", events)

    print("\nCreating 10 concurrent orders to test concurrency...")

    async def create_and_monitor(index: int):
        customer_data = {
            "name": f"Customer {index}",
            "email": f"customer{index}@example.com",
            "address": f"{index} Concurrency Lane"
        }
        items = [
            {"name": f"Item{index}", "price": round(10 + index * 2.5, 2),\
             "quantity": 2}
        ]
        order_response = await system.process_order(customer_data, items)
        order_id = order_response["order_id"]
        max_attempts = 100
        attempts = 0
        while attempts < max_attempts:
            status_response = await system.get_order_status(order_id)
            if not status_response["success"]:
                break
            order_status = status_response["order"]["status"]
            if order_status in ["completed", "failed"]:
                break
            await asyncio.sleep(0.1)
            attempts += 1
        final_status = await system.get_order_status(order_id)
        print(
            f"[Order {index}, Order ID: {order_id}] Final status: "
            f"{final_status['order']['status'].capitalize()}"
        )

    tasks = [asyncio.create_task(create_and_monitor(i)) for i in range(1, 11)]
    await asyncio.gather(*tasks)

    health_status = await system.get_health()
    print("\nSystem Health before shutdown:", health_status)
    await system.shutdown()
    print("System has been shutdown gracefully.")
    health_status_post_shutdown = await system.get_health()
    print("System Health after shutdown:", health_status_post_shutdown)


def main():
    asyncio.run(driver())


if __name__ == "__main__":
    main()


In [ ]:
# tests

import unittest
import asyncio
from unittest.mock import patch
from main import OrderProcessingSystem, TimeProvider


class TestOrderProcessingSystem(unittest.IsolatedAsyncioTestCase):
    """Unit tests for OrderProcessingSystem."""

    def setUp(self):
        self.time_provider = TimeProvider()
        self.system = OrderProcessingSystem(time_provider=self.time_provider)

    async def asyncSetUp(self):
        await self.system.startup()

    async def asyncTearDown(self):
        await self.system.shutdown()

    async def wait_for_order_completion(self, order_id, timeout=5.0):
        """Helper method to wait for order to reach a final state."""
        start_time = asyncio.get_event_loop().time()
        while asyncio.get_event_loop().time() - start_time < timeout:
            status = await self.system.get_order_status(order_id)
            if (
                status["success"]
                and status["order"]["status"] in ["completed", "failed"]
            ):
                return status["order"]["status"]
            await asyncio.sleep(0.1)
        # Return current status if timeout reached
        status = await self.system.get_order_status(order_id)
        return status["order"]["status"] if status["success"] else "unknown"

    async def test_concurrent_orders_processing(self):
        """Ensure system handles multiple concurrent orders correctly."""
        tasks = [
            self.system.process_order(
                customer_data={
                    "name": f"User{i}",
                    "email": f"u{i}@x.com",
                    "address": str(i),
                },
                items=[{"name": "Item", "price": i * 10.0, "quantity": 1}],
            )
            for i in range(1, 6)
        ]
        results = await asyncio.gather(*tasks)

        # Wait for all orders to complete
        completed_count = 0
        for result in results:
            final_status = await self.wait_for_order_completion(
                result["order_id"])
            if final_status == "completed":
                completed_count += 1

        # At least most orders should complete successfully
        self.assertGreaterEqual(completed_count, 3)

    async def test_order_validation_failure(self):
        """Trigger a validation failure and ensure error handling works."""
        with patch.object(
            self.system, "validate_order", side_effect=Exception("Invalid")
        ):
            order = await self.system.process_order(
                customer_data={
                    "name": "Invalid",
                    "email": "inv@example.com",
                    "address": "",
                },
                items=[{"name": "Test", "price": 10.0, "quantity": 1}],
            )

            # Wait for order to be processed
            final_status = await self.wait_for_order_completion(
                order["order_id"])
            self.assertEqual(final_status, "failed")

    async def test_resource_cleanup_on_shutdown(self):
        """Ensure async resources are cleaned after shutdown."""
        await self.system.shutdown()
        health = await self.system.get_health()
        self.assertEqual(health["status"], "shutdown")

    async def test_handler_timeout_fallback(self):
        """Test timeout behavior of long-running handlers."""
        order = await self.system.process_order(
            customer_data={"name": "Tom", "email": "tom@x.com", "address": "9"},
            items=[{"name": "Table", "price": 300.0, "quantity": 1}],
        )

        final_status = await self.wait_for_order_completion(order["order_id"])
        self.assertIn(final_status, ["failed", "completed"])

    async def test_get_status_of_nonexistent_order(self):
        """Ensure correct response when querying non-existent order."""
        status = await self.system.get_order_status("invalid_id")
        self.assertFalse(status["success"])

    async def test_get_events_of_nonexistent_order(self):
        """Ensure correct response when querying events for
          non-existent order."""
        events = await self.system.get_order_events("unknown_id")
        self.assertFalse(events["success"])

    async def test_data_consistency_across_states(self):
        """Verify total amount is calculated correctly through all states."""
        order = await self.system.process_order(
            customer_data={
                "name": "Sam", "email": "sam@example.com", "address": "22"},
            items=[{"name": "Shoe", "price": 50.0, "quantity": 2}],
        )

        await self.wait_for_order_completion(order["order_id"])

        status = await self.system.get_order_status(order["order_id"])
        self.assertEqual(status["order"]["total_amount"], 100.0)

    async def test_shutdown_with_pending_events(self):
        """Test graceful shutdown while events are pending."""
        task = asyncio.create_task(
            self.system.process_order(
                customer_data={
                    "name": "Grace",
                    "email": "grace@example.com",
                    "address": "z",
                },
                items=[{"name": "Clock", "price": 75.0, "quantity": 1}],
            )
        )
        await asyncio.sleep(0.1)
        await self.system.shutdown()
        self.assertTrue(True)

    async def test_compensation_logic_invoked_on_failure(self):
        """Check that compensation logic is invoked when order fails."""
        with patch.object(
            self.system, "process_payment", side_effect=Exception(
                "Payment failed")
        ):
            order = await self.system.process_order(
                customer_data={
                    "name": "Comp",
                    "email": "c@e.com",
                    "address": "9",
                },
                items=[{"name": "Watch", "price": 199.99, "quantity": 1}],
            )

            final_status = await self.wait_for_order_completion(
                order["order_id"])
            self.assertEqual(final_status, "failed")

    async def test_health_check_endpoint(self):
        """Ensure health check reflects system readiness."""
        health = await self.system.get_health()
        self.assertTrue(health["success"])
        self.assertEqual(health["status"], "healthy")

    async def test_event_logging_with_correlation_ids(self):
        """Verify that correlation IDs are present in logged events."""
        order = await self.system.process_order(
            customer_data={
                "name": "Log",
                "email": "log@example.com",
                "address": "logging",
            },
            items=[{"name": "Disk", "price": 55.0, "quantity": 2}],
        )

        await self.wait_for_order_completion(order["order_id"])

        events = await self.system.get_order_events(order["order_id"])
        for event in events["events"]:
            self.assertIn("timestamp", event)
            self.assertIsInstance(event["data"], dict)



# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/67666/view

#### Model code:

```python
import asyncio
import uuid
from typing import List, Dict
from datetime import datetime, timezone

class TimeProvider:
    @staticmethod
    def now():
        return datetime.now(timezone.utc).isoformat()

class OrderProcessingSystem:
    def __init__(self, time_provider: TimeProvider = None):
        self.time_provider = time_provider or TimeProvider()
        self.orders = {}
        self.event_queues = {}
        self.order_lock = asyncio.Lock()

    # <Issue>: Model's code does not yield any output
    async def process_order(self, customer_data: dict, items: List[dict]) -> dict:
        order_id = str(uuid.uuid4())
        total_amount = sum(item["price"] * item["quantity"] for item in items)
        
        order = {
            "order_id": order_id,
            "status": "pending",
            "created_at": self.time_provider.now(),
            "updated_at": self.time_provider.now(),
            "total_amount": total_amount
        }

        event = {
            "event_type": "ORDER_CREATED",
            "timestamp": self.time_provider.now(),
            "data": customer_data,
            "processed": False
        }

        async with self.order_lock:
            self.orders[order_id] = order
            self.event_queues[order_id] = [event]

        await self.handle_event(order_id, event)
        # <Issue>: Model is using synchronous event processing instead of async event-driven architecture

        return {
            "success": True,
            "order_id": order_id,
            "message": "Order processed successfully",
            "timestamp": self.time_provider.now()
        }

    async def handle_event(self, order_id: str, event: dict):
        # Simplified event handling - should include transition checks
        # Assume perfect ordering of events for this implementation
        await asyncio.sleep(1)  # Simulate processing time
        async with self.order_lock:
        # <Issue>: Global lock prevents any concurrency and creates severe bottleneck
            order = self.orders[order_id]
            if order["status"] == "pending":
                order["status"] = "validated"
            elif order["status"] == "validated":
                order["status"] = "payment_processed"
            elif order["status"] == "payment_processed":
                order["status"] = "shipped"
            elif order["status"] == "shipped":
                order["status"] = "completed"
            # <Issue>: Missing state machine validation and invalid transition prevention

            order["updated_at"] = self.time_provider.now()
            event["processed"] = True
            new_event = {
                "event_type": f"ORDER_{order['status'].upper()}",
                "timestamp": self.time_provider.now(),
                "data": event["data"],
                "processed": False
            }
            # <Issue>: Creates incorrect event types like "ORDER_PAYMENT_PROCESSED" instead of "PAYMENT_PROCESSED"

            self.event_queues[order_id].append(new_event)

            if order["status"] != "completed":
                await self.handle_event(order_id, new_event)
                # <Issue>: Recursive calls cause stack overflow and infinite loops

    async def get_order_status(self, order_id: str) -> dict:
        async with self.order_lock:
            # <Issue>: Unnecessary locking for read operations reduces performance
            order = self.orders.get(order_id, None)
            if order is None:
                return {"success": False, "message": "Order not found"}
                # <Issue>: Inconsistent response format missing "order" field

            return {
                "success": True,
                "order": order
            }

    async def get_order_events(self, order_id: str) -> dict:
        async with self.order_lock:
            # <Issue>: Unnecessary locking for read operations reduces performance
            events = self.event_queues.get(order_id, None)
            if events is None:
                return {"success": False, "message": "Order not found"}
                # <Issue>: Inconsistent response format missing "events" field

            return {
                "success": True,
                "events": events
            }

    async def get_health(self) -> dict:
        # a simplified health check example
        return {"status": "ok"}

    async def startup(self):
        # perform any start-up operations if needed
        pass
        # <Issue>: Model did not implement any startup logic
    
    async def shutdown(self):
        # clean up resources or save states if necessary
        pass
        # <Issue>: Model did not implement any shutdown logic
    
    # <Issue>: Model's implementation is missing event deduplication leading to potential duplicate processing

# Example of usage
async def main():
    system = OrderProcessingSystem()
    await system.startup()

    input_data = {
        "customer_data": {
            "name": "John Doe",
            "email": "john@example.com",
            "address": "123 Main St"
        },
        "items": [
            {"name": "Laptop", "price": 999.99, "quantity": 1}
        ]
    }

    order_response = await system.process_order(input_data["customer_data"], input_data["items"])
    print(order_response)

    order_status = await system.get_order_status(order_response["order_id"])
    print(order_status)

    order_events = await system.get_order_events(order_response["order_id"])
    print(order_events)

    # Shut down system
    await system.shutdown()

# Run the example driver function
asyncio.run(main())

```